# YOLOE для bulk-разметки логотипов Т-Банка в Google Colab

Этот notebook предназначен для автоматической разметки подмножества data_sirius (1000 изображений) с использованием модели YOLOE.
Цель: детекция 3 классов логотипов Т-Банка — yellow_shield_black_T, white_shield_black_T, purple_shield_white_T.

Используются гибридные промпты (визуальные из референсов и текстовые описания).
Выход: pseudo_coco.json с псевдо-аннотациями и ZIP с предсказаниями.

**Предварительные требования:**
- Подготовьте refs_ls_coco.json (COCO-формат bbox для 9 референсных изображений, классы 0-2).
- Подготовьте ZIP-архив data_sirius_subset.zip с 1000 изображениями.

In [ ]:
!pip install -U ultralytics pycocotools opencv-python pillow numpy -q

## 2. Загрузка референсных аннотаций

Загрузите файл refs_ls_coco.json — экспорт из Label Studio в COCO-формате с bbox для 9 референсных изображений (группированные по классам 0-2).

In [ ]:
from google.colab import files
uploaded = files.upload()

import json
with open('refs_ls_coco.json', 'r') as f:
    refs_data = json.load(f)
print('Референсные данные загружены.')

## 3. Загрузка подмножества data_sirius

Загрузите ZIP-архив с 1000 изображениями и разархивируйте в /content/data_sirius_subset/.

In [ ]:
uploaded = files.upload()  # Загрузите data_sirius_subset.zip
!unzip data_sirius_subset.zip -d /content/data_sirius_subset/
print('Данные разархивированы в /content/data_sirius_subset/')

## 4. Загрузка модели YOLOE

Загружаем предобученную модель YOLOE для сегментации (yoloe-11l-seg.pt). Модель скачается автоматически, если не существует.

In [ ]:
from ultralytics import YOLOE
model = YOLOE('yoloe-11l-seg.pt')
print('Модель загружена.')

## 5. Подготовка гибридных промптов

Визуальные промпты: извлекаем bbox и cls из refs_ls_coco.json для классов 0-2, нормализуем координаты.
Текстовые промпты: описания классов для zero-shot компоненты.

In [ ]:
# Извлечение визуальных промптов из референсов
ref_images_dict = {img['id']: img for img in refs_data['images']}
ref_annotations = [ann for ann in refs_data['annotations'] if 0 <= ann['category_id'] <= 2]
bboxes = []
cls_list = []
for ann in ref_annotations:
    img = ref_images_dict[ann['image_id']]
    x, y, w, h = ann['bbox']
    x_norm = x / img['width']
    y_norm = y / img['height']
    w_norm = w / img['width']
    h_norm = h / img['height']
    bboxes.append([x_norm, y_norm, w_norm, h_norm])
    cls_list.append(ann['category_id'])
visual_prompts = {'bboxes': bboxes, 'cls': cls_list}
print(f'Извлечено {len(bboxes)} визуальных промптов.')

In [ ]:
# Определение классов и текстовых описаний
class_names = ['yellow_shield_black_T', 'white_shield_black_T', 'purple_shield_white_T']
text_descriptions = [
    'yellow shield with black T logo',
    'white shield with black T logo',
    'purple shield with white T logo'
]
# Если API YOLOE поддерживает, установить классы и текст (раскомментировать при необходимости)
# model.set_classes(class_names, text_prompts=text_descriptions)  # или аналогично get_text_pe

## 6. Построение списка изображений для экспорта COCO

In [ ]:
import os
from PIL import Image
image_dir = '/content/data_sirius_subset/'
image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
coco_images = []
for i, fname in enumerate(image_files):
    img_path = os.path.join(image_dir, fname)
    with Image.open(img_path) as im:
        w, h = im.size
    coco_images.append({'id': i, 'width': w, 'height': h, 'file_name': fname})
print(f'Подготовлено {len(coco_images)} изображений.')

## 7. Предсказание на изображениях

Запускаем inference с гибридными промптами, conf=0.5, iou=0.7. Сохраняем TXT и изображения в runs/colab_predict.

In [ ]:
results = model.predict(
    source=image_dir,
    visual_prompts=visual_prompts,
    # refer_image='example_ref.jpg',  # раскомментировать, если нужен референсный пример
    # text_prompts=text_descriptions,  # если поддерживается
    conf=0.5,
    iou=0.7,
    save_txt=True,
    save=True,
    project='runs/colab_predict',
    device='cuda' if available else 'cpu'  # автоматический
)
print('Предсказание завершено.')

## 8. Экспорт предсказаний в COCO-формат

Конвертируем результаты в pseudo_coco.json: bbox в xywh (пиксели), category_id 0-2, score. Используем объекты results для доступа к conf.

In [ ]:
categories = [{'id': i, 'name': class_names[i]} for i in range(3)]
annotations = []
ann_id = 0
for i, result in enumerate(results):
    if result.boxes is not None and len(result.boxes) > 0:
        boxes = result.boxes.xywh.cpu().numpy()
        clss = result.boxes.cls.cpu().numpy().astype(int)
        confs = result.boxes.conf.cpu().numpy()
        img_w = coco_images[i]['width']
        img_h = coco_images[i]['height']
        for j in range(len(clss)):
            x_c, y_c, bw, bh = boxes[j]
            x_min = (x_c - bw / 2) * img_w
            y_min = (y_c - bh / 2) * img_h
            w_pix = bw * img_w
            h_pix = bh * img_h
            area = w_pix * h_pix
            annotations.append({
                'id': ann_id,
                'image_id': i,
                'category_id': int(clss[j]),
                'bbox': [float(x_min), float(y_min), float(w_pix), float(h_pix)],
                'area': float(area),
                'iscrowd': 0,
                'score': float(confs[j])
            })
            ann_id += 1
pseudo_coco = {
    'images': coco_images,
    'annotations': annotations,
    'categories': categories
}
with open('pseudo_coco.json', 'w') as f:
    json.dump(pseudo_coco, f, indent=2)
print(f'Сохранено {len(annotations)} аннотаций для {len(coco_images)} изображений в pseudo_coco.json.')

## 9. Скачивание результатов

Скачиваем pseudo_coco.json и ZIP с runs/colab_predict (изображения с предсказаниями и TXT).

In [ ]:
from google.colab import files
files.download('pseudo_coco.json')
!zip -r runs_colab.zip runs/colab_predict/
files.download('runs_colab.zip')

## Опционально: Оценка mAP на sample val

Загрузите небольшой GT-файл (small_val.json) для вычисления mAP с использованием pycocotools.
Это позволит оценить качество псевдо-разметки на валидационной выборке.

In [ ]:
uploaded_gt = files.upload()  # Загрузите small_val.json
# Предполагаем, что файл называется 'small_val.json'
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
gt_coco = COCO('small_val.json')
pred_coco = gt_coco.loadRes('pseudo_coco.json')
coco_eval = COCOeval(gt_coco, pred_coco, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
print('Оценка mAP завершена.')